In [2]:
#import dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [3]:
# Set device to CUDA if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == 'cuda':
    try:
        print("GPU:", torch.cuda.get_device_name(0))
        torch.backends.cudnn.benchmark = True
    except Exception as e:
        print("Could not query GPU name:", e)


Using device: cuda
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [4]:
#transforms
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5), (0.5))
                              ])

In [5]:
#load data
train_data=datasets.FashionMNIST(root='./data',
                             train=True,
                             download=False,
                             transform=transform)
test_data=datasets.FashionMNIST(root='./data',
                            train=False,
                            download=False,
                            transform=transform)

train_loader=DataLoader(train_data,
                        batch_size=128,
                        shuffle=True)
test_loader=DataLoader(test_data,
                       batch_size=128,
                       shuffle=False)

In [6]:
#architecture
class cnn_fashionmnist(nn.Module):
  def __init__(self):
    super(cnn_fashionmnist,self).__init__()
    self.conv1=nn.Conv2d(1,32,3,padding=1)#convolution layer
    self.pool1=nn.MaxPool2d(2,2)#pooling layer

    self.conv2=nn.Conv2d(32,64,3)#convolution layer
    self.pool2=nn.MaxPool2d(2,2)#pooling layer

    #FahionMNIST size is 28X28X1
    #Converted to 14X14X1
    #Converted to 14X14X64
    #Then to 7X7X64
    #Dense layers
    self.fc1=nn.Linear(6*6*64,256)
    self.fc2=nn.Linear(256,128)
    self.fc3=nn.Linear(128,10)

  def forward(self,x):
    #x represents the image input
    x=self.conv1(x)
    x=torch.relu(x)
    x=self.pool1(x)

    x=self.pool2(torch.relu(self.conv2(x)))
    #x=x.Flatten(1)#x=x.view(x.size(0),-1)
    x=x.view(x.size(0),-1) # Flatten the tensor
    x=self.fc1(x)
    x=torch.relu(x)
    x=self.fc2(x)
    x=torch.relu(x)
    x=self.fc3(x)
    return x

In [7]:
#object creation
model=cnn_fashionmnist().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),
                     lr=0.001)

In [8]:
#train
for epoch in range(5):
  for image,label in train_loader:
    output=model(image)
    loss=criterion(output,label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print(f"epoch:{epoch+1},loss:{loss.item}")

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for images,labels in test_loader:
    images,labels=images.to(device),labels.to(device)
    outputs=model(images)
    max,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
  print(f"Test Accuracy: {100*correct/total}%")